Created on 4/2/2020
Author: Yuan-Chi Yang

Objective: template for content analysis on political feedback, please feel free to modify and play around

In [23]:
import pandas as pd
import nltk
import re

# Importing the data and perform some checks

The data is from "C:\Users\yyang60\PostDoct-Emory\medicaid-project\medicaid-classifier\labeling-data\whole_dataset\BERT_20200228\political-tweets-streaming.csv"

It consists of all the tweets classified as the 'p' class by the best performing classfiers to date.

In [24]:
df = pd.read_csv('./political-tweets-streaming.csv',header = 0, keep_default_na=False,dtype={'tweet_id':str})

In [25]:
len(df)

383969

In [26]:
df.columns

Index(['tweet_id', 'unprocessed_text', 'class'], dtype='object')

### Check Duplicates
No Duplicated

In [27]:
df.duplicated(subset = ['tweet_id'], keep=False).sum()

0

### Include 'text_remove_stopwords' column

In [28]:
def loadStopWords(FILENAME):
    stopword_list = []
    infile = open(FILENAME)
    for line in infile:
        stopword_list.append(line.strip())
    print(len(stopword_list))
    return stopword_list

In [29]:
def processing_text_remove_stopwords(tweet_text,stop_words):
    tweet_text = re.sub(r'&amp;', "and", tweet_text)
    tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',
                        tweet_text)
    #tweet_text = re.sub('(@[A-Za-z0-9\_]+)', '', tweet_text)
    tweet_text = re.sub('(@[\S]+)', '', tweet_text)#added Angel
    ####the following section separate words following each hashtag
    list_hashtag1=re.findall('(#[\S]+)',tweet_text)
    list_hashtag2=[]
    for p in list_hashtag1:
        p=re.sub('(#)',' ',p)#clear hashtag symbols
        p1=re.findall('([A-Z]{2,})',p)#uppercase abbrevs
        for p2 in p1:
            list_hashtag2.append(p2)#add abbrevs
        p=re.sub('([A-Z]{2,})',' ',p)#clear uppercase abbrevs
        p1=re.findall('([A-Z][a-z]{1,})',p)#words start uppercase letter
        for p2 in p1:
            list_hashtag2.append(p2)
        p=re.sub('([A-Z][a-z]{1,})',' ',p)#clear words start with uppercase letter
        p=re.sub('([A-Z])',' ',p)#clear single uppercase letters
        p=re.sub('([^A-Za-z])',' ',p)#clear symbols
        p1=p.split()#find leftover words
        for p2 in p1:
            list_hashtag2.append(p2)#add leftover words
    tweet_text=re.sub('(#[\S]+)','',tweet_text)
    
    tweet_text = re.sub("[^a-zA-Z_-]", " ", tweet_text)
    tweet_text = tweet_text.lower()
    tweet_text = re.sub(r'\s{2,}', " ", tweet_text)
    list_hashtag= [h for h in list_hashtag2 if (not h in stop_words and len(h)>1)]
    tweet_text = [t for t in tweet_text.split() if (not t in stop_words and len(t)>1)]
    tweet_text.extend(list_hashtag)
    return ' '.join(tweet_text)

In [30]:
stopwords = set(loadStopWords('./stopwords.txt'))

261


In [31]:
df['text_remove_stopwords'] = df['unprocessed_text'].apply(lambda x: processing_text_remove_stopwords(x,stopwords))

## Find the most frequent words in text

In [10]:
unigrams = ' '.join(df['text_remove_stopwords'].to_list()).split()
bigrams = []
for tweet in df['text_remove_stopwords'].to_list():
    bigrams += list(nltk.bigrams(tweet.split()))
    
trigrams = []
for tweet in df['text_remove_stopwords'].to_list():
    trigrams += list(nltk.trigrams(tweet.split()))
uni_fd = nltk.FreqDist(unigrams)
big_fd = nltk.FreqDist(bigrams)
trig_fd = nltk.FreqDist(trigrams)


In [ ]:
def wordInNumTweets(s):
    pattern = rf'(^|[^a-zA-Z]){s}([^a-zA-Z]|$)' #rf is for using a variable inside
    count=0
    for i in range(len(df['text_remove_stopwords'])):
         if re.search(pattern,df['text_remove_stopwords'].iloc[i]) is not None :
            count=count+1
    return count

In [ ]:
num = 40

print('unigrams:\n')
fd = uni_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
df_term1=[]
df_count1=[]
df_numtweets1=[]
for i in range(0,num):
    print(fd_list[i][0], fd_list[i][1])
    df_term1.append(fd_list[i][0])
    df_count1.append(fd_list[i][1])
    df_numtweets1.append(wordInNumTweets(fd_list[i][0]))

print('--------------------------------------------------------\n')
print('bigrams:\n')
fd = big_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
df_term2=[]
df_count2=[]
df_numtweets2=[]
for i in range(0,num):
    x, y= fd_list[i][0]
    term = x + ' '+ y
    print(term, fd_list[i][1])
    df_term2.append(term)
    df_count2.append(fd_list[i][1])
    df_numtweets2.append(wordInNumTweets(term))

print('--------------------------------------------------------\n')
print('trigrams:\n')
fd = trig_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
df_term3=[]
df_count3=[]
df_numtweets3=[]
for i in range(0,num):
    x, y, z= fd_list[i][0]
    term = x + ' '+ y + ' ' + z
    print(term, fd_list[i][1])
    df_term3.append(term)
    df_count3.append(fd_list[i][1])
    df_numtweets3.append(wordInNumTweets(term))
df_ubt={'uni_term':df_term1,'uni_count':df_count1,'uni_numtw':df_numtweets1,'bi_term':df_term2,'bi_count':df_count2,'bi_numtw':df_numtweets2,'tri_term':df_term3,'tri_count':df_count3,'tri_numtw':df_numtweets3}
df_ubt=pd.DataFrame(df_ubt)
df_ubt.to_csv('ubt40.csv')

unigrams:

people 66954
social 64162
security 59281
care 51287
get 48687
health 41356
would 40135
pay 39945
insurance 39921
trump 39077
healthcare 38641
expansion 38198
like 35822
cut 30612
tax 29473
state 29422
cuts 28059
need 26424
want 26282
food 23709
work 23698
programs 23589
us 23326
re 23125
money 22908
states 21239
aca 20763
free 20572
one 19826
know 19507
also 19277
take 19129
government 19113
poor 19012
public 18829
many 18774
stamps 18513
budget 18252
coverage 17871
make 17382
--------------------------------------------------------

bigrams:

social security 56343
food stamps 18233
health care 16441
health insurance 8651
tax cuts 6999
private insurance 5939
cut social 5701
cuts social 4495
take away 4110
work requirements 3636
middle class 3528
low income 3129
mental health 3037
budget cuts 2976
pre-existing conditions 2964
Medicare Medicaid 2922
trump budget 2918
For All 2791
insurance companies 2751
tax cut 2736
poor people 2705
public schools 2632


## Add sentiment scores to all tweets

In [36]:
print(df['polarity'].iloc[1])

0.2


In [37]:
print(df['polarity'].iloc[111])

0.0


In [41]:
print(df['polarity'].iloc[311])

0.2857142857142857


In [32]:
from textblob import TextBlob
def addpolarityscores(text):
    t=TextBlob(text)
    return t.sentiment.polarity
def addsubjectivityscores(text):
    t=TextBlob(text)
    return t.sentiment.subjectivity

In [42]:
df['polarity'] = df['text_remove_stopwords'].apply(lambda x:addpolarityscores(x))
df['subjectivity'] = df['text_remove_stopwords'].apply(lambda x:addsubjectivityscores(x))

## Add label of 1/0 based on the existence/absence of each interesting term

In [71]:
def highfreqword(text,terms):
    exist=0
    for j in range(len(terms)):
        pattern = rf'(^|[^a-zA-Z]){terms[j]}([^a-zA-Z]|$)' #rf is for using a variable inside
        if(re.search(pattern,text)!=None):
            exist=1
    if exist==1:
        return 1
    else:
        return 0

In [43]:
term_ls=[['cut social security','cutting social security','cuts social security','social security cuts'],['mental health'],['middle class'],['affordable care act','aca'],['tax cut','tax cuts'],['food stamps'],['low income'],['planned parenthood'],['minimum wage'],['illegal immigrants']]
for i in range(10):
    label_name=(term_ls[i])[0]
    df[label_name] = df['text_remove_stopwords'].apply(lambda x:highfreqword(x,term_ls[i]))

In [129]:
df_temp=df[df['middle class'].apply(lambda x:x==1)]
len(df_temp)

3382

In [132]:
for i in range(16):
    df_temp1=df_temp[df[bigrams_ls[i]].apply(lambda x:x==1)]
    print(len(df_temp1))
df_temp1=df_temp[df['healthcare'].apply(lambda x:x==1)]

<ipython-input-132-7e03087d1343>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp1=df_temp[df[bigrams_ls[i]].apply(lambda x:x==1)]
<ipython-input-132-7e03087d1343>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp1=df_temp[df['healthcare'].apply(lambda x:x==1)]


862
11
458
385
125
157
100
47
108
75
8
31
43
18
144
28


In [54]:
#term2 = 'trump budget'
#pattern = rf'(^|[^a-zA-Z]){term2}([^a-zA-Z]|$)' #rf is for using a variable inside
#df_temp2 = df[df['text_remove_stopwords'].apply(lambda x: re.search(pattern,x)!=None)]
#print('# of tweets:',len(df_temp2))


In [55]:
#df_temp12 =df_temp1.append(df_temp2)
#print('# of duplicated tweets:',df_temp12.duplicated(subset = ['tweet_id'], keep=False).sum())

# of duplicated tweets: 0


In [136]:
num = 30
temp = df_temp1.sample(n=num)
#for i in range(0,num):
    #print(temp['unprocessed_text'].iloc[i],'\n\n')
    #print(temp['text_remove_stopwords'].iloc[i],'\n\n')
    #print('---------------------------------------------------------------------------------')

In [137]:
#temp.to_csv('ss_pec_sample.csv')
#temp.to_csv('mh_e_sample.csv')
temp.to_csv('mc_h_sample.csv')

